In [10]:
import numpy as np
import rdkit.Chem as Chem
import sys
import os
sys.path.append("../../")

import h5py
from tqdm import tqdm
from src.data.utils import pdb_to_rdkit_mol, mol2_to_rdkit_mol
from src.data.utils import rdkit_mol_featurizer
from src.data.pocket_utils import combine_and_filter, remove_waters

In [11]:
def extend_protein_data_dict(protein_data, protein_name, vdw_graph, ionic_graph, cov_graph):
    protein_data[protein_name] = {}
    protein_data[protein_name]['vdw'] = {}
    protein_data[protein_name]['vdw']['node_features'] = vdw_graph[0]
    protein_data[protein_name]['vdw']['edge_features'] = vdw_graph[1]
    protein_data[protein_name]['vdw']['edge_list'] = vdw_graph[2]

    protein_data[protein_name]['ionic'] = {}
    protein_data[protein_name]['ionic']['node_features'] = ionic_graph[0]
    protein_data[protein_name]['ionic']['edge_features'] = ionic_graph[1]
    protein_data[protein_name]['ionic']['edge_list'] = ionic_graph[2]

    protein_data[protein_name]['cov'] = {}
    protein_data[protein_name]['cov']['node_features'] = cov_graph[0]
    protein_data[protein_name]['cov']['edge_features'] = cov_graph[1]
    protein_data[protein_name]['cov']['edge_list'] = cov_graph[2]

    return protein_data

# Function to save data to HDF5
def save_protein_data_to_hdf5(file_name, protein_data):
    with h5py.File(file_name, 'w') as f:
        for protein, graphs in protein_data.items():
            protein_group = f.create_group(protein)
            for graph_name, graph_data in graphs.items():
                graph_group = protein_group.create_group(graph_name)
                for data_name, data in graph_data.items():
                    graph_group.create_dataset(data_name, data=data)

def load_protein_data_from_hdf5(file_name):
    with h5py.File(file_name, 'r') as f:
        protein_data = {}
        for protein in f.keys():
            protein_data[protein] = {}
            for graph_name in f[protein].keys():
                protein_data[protein][graph_name] = {
                    'node_features': f[protein][graph_name]['node_features'][:],
                    'edge_features': f[protein][graph_name]['edge_features'][:],
                    'edge_list': f[protein][graph_name]['edge_list'][:]
                }
    return protein_data


In [12]:
# protein_data = {
#     'protein_1': {
#         'graph_1': {
#             'node_features': np.random.rand(5, 10),
#             'edge_features': np.random.rand(4, 5),
#             'edge_list': np.array([(0, 1), (1, 2), (2, 3), (3, 4)])
#         },
#         'graph_2': {
#             'node_features': np.random.rand(5, 10),
#             'edge_features': np.random.rand(4, 5),
#             'edge_list': np.array([(0, 1), (1, 2), (2, 3), (3, 4)])
#         },
#         'graph_3': {
#             'node_features': np.random.rand(5, 10),
#             'edge_features': np.random.rand(4, 5),
#             'edge_list': np.array([(0, 1), (1, 2), (2, 3), (3, 4)])
#         }
#     },
#     # Add more proteins as needed
# }
protein_data = {}

In [13]:
#define a timer 
from time import time
global t0
t0 = time()
def time_block():    
    global t0 
    print(f"Time elapsed: {time()-t0:.2f}")
    t0 = time()

In [5]:
from time import time
import os
from tqdm import tqdm
from rdkit import Chem
from contextlib import contextmanager

@contextmanager
def time_block(label):
    start = time()
    try:
        yield
    finally:
        end = time()
        print(f"{label}: {end - start:.2f} seconds")

# Specify the path to the PDB data
#pdb_data = '/Users/tsachmackey/dfs/affinity_net/PDBbind/v2020-other-PL'
pdb_data = '../../test_data/pdb'

# Initialize the protein data dictionary
protein_data = {}

for filename in tqdm(os.listdir(pdb_data)):
    if os.path.isdir(os.path.join(pdb_data, filename)):
        protein_name = filename

        ligand_mol2_path = os.path.join(pdb_data, f'{protein_name}/{protein_name}_ligand.mol2')
        protein_pdb_path = os.path.join(pdb_data, f'{protein_name}/{protein_name}_protein.pdb')

        with time_block("Time for loading files"):
            ligand_mol = mol2_to_rdkit_mol(ligand_mol2_path, sanitize=False)
            protein_mol = pdb_to_rdkit_mol(protein_pdb_path, sanitize=False)
            
        # Set by_residue to True to include entire residues
        with time_block("Time for combining and filtering"):
            pocket_mol_res = combine_and_filter(ligand_mol, remove_waters(protein_mol), by_residue=True)

        # Save the resulting complex_mol to a PDB file
        output_path = os.path.join(pdb_data, f'{protein_name}/{protein_name}_pocket_res.pdb')
        pdb_block = Chem.MolToPDBBlock(pocket_mol_res)

        with time_block("Time for writing to PDB"):
            with open(output_path, 'w') as file:
                file.write(pdb_block)

        with time_block("Time for featurizing"):
            vdw_graph = rdkit_mol_featurizer(pocket_mol_res, "vdw interactions")
            ionic_graph = rdkit_mol_featurizer(pocket_mol_res, "ionic interactions")
            cov_graph = rdkit_mol_featurizer(pocket_mol_res, "covalent bonds")

        protein_data = extend_protein_data_dict(protein_data, protein_name, vdw_graph, ionic_graph, cov_graph)

# Save the example data
save_protein_data_to_hdf5('protein_data.h5', protein_data)


  0%|          | 0/9 [00:00<?, ?it/s]

Time for loading files: 0.01 seconds


 11%|█         | 1/9 [00:00<00:05,  1.43it/s]

Time for combining and filtering: 0.48 seconds
Time for writing to PDB: 0.00 seconds
Time for featurizing: 0.20 seconds
Time for loading files: 0.06 seconds


 44%|████▍     | 4/9 [00:02<00:02,  1.74it/s]

Time for combining and filtering: 1.47 seconds
Time for writing to PDB: 0.00 seconds
Time for featurizing: 0.10 seconds
Time for loading files: 0.03 seconds


100%|██████████| 9/9 [00:03<00:00,  2.92it/s]

Time for combining and filtering: 0.56 seconds
Time for writing to PDB: 0.00 seconds
Time for featurizing: 0.16 seconds


In [6]:
# Load the example data
loaded_protein_data = load_protein_data_from_hdf5('protein_data.h5')

In [7]:
loaded_protein_data

{'1a0q': {'cov': {'node_features': array([[-1.   ,  7.   , 14.007, ...,  1.   ,  0.   ,  0.   ],
          [-1.   ,  1.   ,  1.008, ...,  0.   ,  0.   ,  0.   ],
          [-1.   ,  6.   , 12.011, ...,  1.   ,  0.   ,  0.   ],
          ...,
          [ 1.   ,  1.   ,  1.008, ...,  0.   ,  0.   ,  0.   ],
          [ 1.   ,  1.   ,  1.008, ...,  0.   ,  0.   ,  0.   ],
          [ 1.   ,  1.   ,  1.008, ...,  0.   ,  0.   ,  0.   ]]),
   'edge_features': array([[1., 0., 0., 0., 0.],
          [1., 0., 0., 0., 0.],
          [1., 0., 0., 0., 0.],
          ...,
          [1., 0., 0., 0., 0.],
          [1., 0., 0., 0., 0.],
          [1., 0., 0., 0., 0.]]),
   'edge_list': array([[378, 375],
          [375, 378],
          [814, 811],
          ...,
          [994, 969],
          [977, 995],
          [995, 977]])},
  'ionic': {'node_features': array([[-1.   ,  7.   , 14.007, ...,  1.   ,  0.   ,  0.   ],
          [-1.   ,  1.   ,  1.008, ...,  0.   ,  0.   ,  0.   ],
          [-1.  

In [8]:
os.remove('protein_data.h5')

In [14]:
pdb_data = '/Users/tsachmackey/dfs/affinity_net/PDBbind/refined-set'
new_pdb_data_dir = '/Users/tsachmackey/dfs/affinity_net/PDBbind/refined-set_featurized'

# Initialize the protein data dictionary
protein_data = {}

for filename in tqdm(os.listdir(pdb_data)):
    if os.path.isdir(os.path.join(pdb_data, filename)):
        #make a new directory in the new_pdb_data_dir
        protein_name = filename
        new_protein_dir = os.path.join(new_pdb_data_dir, protein_name)
        os.makedirs(new_protein_dir, exist_ok=True)

        #copy {protein_name}_ligand.mol2, {protein_name}_protein.pdb, {protein_name}_pocket_res.pdb, and protein_data.h5
        #to the new_protein_dir
        os.system(f"cp {pdb_data}/{protein_name}/{protein_name}_ligand.mol2 {new_protein_dir}")
        os.system(f"cp {pdb_data}/{protein_name}/{protein_name}_protein.pdb {new_protein_dir}")
        os.system(f"cp {pdb_data}/{protein_name}/{protein_name}_pocket_res.pdb {new_protein_dir}")
        os.system(f"cp {pdb_data}/{protein_name}/protein_data.h5 {new_protein_dir}")
        
        try: 
            #load in the protein data
            protein_data_i = load_protein_data_from_hdf5(f"{new_protein_dir}/protein_data.h5")
        except:
            print(f"Error loading protein data for {protein_name}")
            continue

        #extend the protein_data dictionary
        protein_data.update(protein_data_i)

#save the data to new_pdb_data_dir/protein_data.h5
save_protein_data_to_hdf5(f"{new_pdb_data_dir}/protein_data.h5", protein_data)

  3%|▎         | 164/5321 [00:02<01:12, 71.31it/s]

Error loading protein data for 3l4y


  4%|▎         | 188/5321 [00:02<01:13, 70.27it/s]cp: /Users/tsachmackey/dfs/affinity_net/PDBbind/refined-set/4kcx/4kcx_pocket_res.pdb: No such file or directory
cp: /Users/tsachmackey/dfs/affinity_net/PDBbind/refined-set/4kcx/protein_data.h5: No such file or directory
  4%|▍         | 203/5321 [00:02<01:13, 69.29it/s]

Error loading protein data for 4kcx


  6%|▌         | 325/5321 [00:04<01:12, 69.22it/s]

...

 95%|█████████▌| 5067/5321 [03:25<00:04, 60.54it/s]

Error loading protein data for 5kva


 98%|█████████▊| 5227/5321 [03:27<00:01, 70.75it/s]

Error loading protein data for 6bm5


100%|██████████| 5321/5321 [03:29<00:00, 25.39it/s]
